In [73]:
import geopandas as gpd
import numpy as np
import os
import pandas as pd

os.chdir("D:\Git Local\CloudSeeding")

In [74]:
df_pt = gpd.read_file("data\cleaned_炮台数据\cleaned_炮台数据.shp")
df_pt = pd.DataFrame(df_pt.drop(columns='geometry'))

In [75]:
gdf_pt = gpd.GeoDataFrame(
    df_pt,
    geometry = gpd.points_from_xy(
        x = df_pt.longitude,
        y = df_pt.latitude,
        crs = 'EPSG:4326', # or: crs = pyproj.CRS.from_user_input(4326)
    )

)

In [79]:
(1.5*1/(0.4**2)+2*(1/97**2))/(1/(0.4**2)+1/(97**2))

1.500008502353026

In [69]:
os.chdir(r"C:\Users\Anora\BFI Dropbox\Wanru Wu\MODIS")
# lat: [-89.5, 89.5]
# lon: [-179.5,179.5]

In [70]:
def get_four_points(longtitude,latitude):

    # round to the nearest 0.5
    lon1 = round(longtitude * 2) / 2
    lat1 = round(latitude * 2) / 2

    # detect if it is an integer
    if lon1.is_integer():
        lon1 = lon1 + 0.5
    if lat1.is_integer():
        lat1 = lat1 + 0.5

    if ((lon1 == longtitude) & (lat1 == latitude)):
        return [(lon1,lat1)]
    
    elif ((lon1 == longtitude) & (lat1 != latitude)):
        if lat1 > latitude:
            lat2 = lat1 - 1
        else:
            lat2 = lat1 + 1
        return [(lon1,lat1),(lon1,lat2)]
    
    elif ((lon1 != longtitude) & (lat1 == latitude)):
        if lon1 > longtitude:
            lon2 = lon1 - 1
        else:
            lon2 = lon1 + 1
        return [(lon1,lat1),(lon2,lat1)]
    
    else:
        if lon1 > longtitude:
            lon2 = lon1 - 1
        else:
            lon2 = lon1 + 1

        if lat1 > latitude:
            lat2 = lat1 - 1
        else:
            lat2 = lat1 + 1
        
        return [(lon1,lat1),(lon1,lat2),(lon2,lat1),(lon2,lat2)]

In [ ]:
from geopy import distance

data = pd.DataFrame(columns=['wt_cloud_optical_thickness', 'wt_cloud_mask_fraction', 
                                     'latitude', 'longitude', 'year', 'month', 'day'])

for index,row in gdf_pt.iterrows():

    # get the points on the coordinate grid
    pts_list  = get_four_points(row["longitude"],row["latitude"])
    print(pts_list)

    cloud_df = pd.DataFrame(columns=['cloud_optical_thickness', 'cloud_mask_fraction', 
                                     'latitude', 'longitude', 'year', 'month', 'day'])
    
    all_weight = 0

    for pts in pts_list:

        print(pts)

        # for each nearest point, get its data for all the days
        pts_df = pd.DataFrame(columns=['cloud_optical_thickness', 'cloud_mask_fraction', 
                                     'latitude', 'longitude', 'year', 'month', 'day'])
        
        for year in range(2011,2024):
            for month in range(1,13):
                
                print(year,month)

                # get the data frame of the points
                temp_df = pd.read_stata(f"{year}\MODIS{year}{month:02}.dta")
                condition = (temp_df['longitude']==pts[0])&(temp_df['latitude']==pts[1])
                temp_df = temp_df[condition]

                # get rid of abnormal date
                if ((year == 2020) & (month == 6)):
                    temp_df = temp_df[temp_df['day']!= 31]

                # concat
                pts_df = pd.concat([pts_df,temp_df])
                del temp_df

        # cloud_df get the data for all four relevant points on all dates
        dist = distance.distance(pts, (row["longitude"],row["latitude"])).km
        pts_df['cloud_optical_thickness'] = pts_df['cloud_optical_thickness']*(1/(dist**2)) 
        pts_df['cloud_mask_fraction']     = pts_df['cloud_mask_fraction']*(1/(dist**2)) 
        all_weight += 1/(dist**2)
        cloud_df = pd.concat([cloud_df,pts_df])
        del pts_df
    
    # for each day, calculate the weighted sum
    new_column_1 = cloud_df.groupby(by = ['year','month','day'])['cloud_optical_thickness'].sum()/all_weight
    new_column_2 = cloud_df.groupby(by = ['year','month','day'])['cloud_mask_fraction'].sum()/all_weight
    cloud_df = cloud_df.set_index(['year','month','day'])

    cloud_df['wt_cloud_optical_thickness'] = new_column_1
    cloud_df['wt_cloud_mask_fraction']     = new_column_2

    cloud_df = cloud_df.reset_index()

    # only keep relevant columns
    cloud_df.drop(columns=['cloud_optical_thickness','cloud_mask_fraction','latitude', 'longitude'],inplace=True)
    cloud_df.drop_duplicates(inplace=True)

    # add information on pt
    cloud_df['latitude'] =  row["latitude"]
    cloud_df['longitude'] = row['longitude']
    data = pd.concat([data,cloud_df])
    del cloud_df
    



In [81]:
gdf_pt

,Unnamed_ 0,weibo_text,operation_,operatio_1,operatio_2,prov,city,county,area,location,coordinate,coordina_1,address,formatted_,first_oper,longitude,latitude,geometry
0,0,勐海县抓住有利天气条件 开展人工增雨作业有效缓解旱情记者黎智仙报道： 3月24日...,2020,3,20,云南省,西双版纳傣族自治州,勐海县,星火山、布朗山、勐混、曼稿保护区、勐邦水库,云南省西双版纳傣族自治州星火山,"100.179358,22.092365",住宅区,西双版纳傣族自治州星火山,云南省西双版纳傣族自治州勐海县星火山,2020-03-20,100.179358,22.092365,POINT (100.17936 22.09236)
1,1,勐海县抓住有利天气条件 开展人工增雨作业有效缓解旱情记者黎智仙报道： 3月24日...,2020,3,20,云南省,西双版纳傣族自治州,勐海县,星火山、布朗山、勐混、曼稿保护区、勐邦水库,云南省西双版纳傣族自治州布朗山,"100.406481,21.626384",村庄,西双版纳傣族自治州布朗山,云南省西双版纳傣族自治州勐海县布朗山,2020-03-20,100.406481,21.626384,POINT (100.40648 21.62638)
2,2,勐海县抓住有利天气条件 开展人工增雨作业有效缓解旱情记者黎智仙报道： 3月24日...,2020,3,20,云南省,西双版纳傣族自治州,勐海县,星火山、布朗山、勐混、曼稿保护区、勐邦水库,云南省西双版纳傣族自治州勐混,"100.390518,21.841447",乡镇,西双版纳傣族自治州勐混,云南省西双版纳傣族自治州勐海县勐混镇,2020-03-20,100.390518,21.841447,POINT (100.39052 21.84145)
3,4,勐海县抓住有利天气条件 开展人工增雨作业有效缓解旱情记者黎智仙报道： 3月24日...,2020,3,20,云南省,西双版纳傣族自治州,勐海县,星火山、布朗山、勐混、曼稿保护区、勐邦水库,云南省西双版纳傣族自治州勐邦水库,"100.290934,21.900714",住宅区,西双版纳傣族自治州勐邦水库,云南省西双版纳傣族自治州勐海县勐邦水库,2020-03-20,100.290934,21.900714,POINT (100.29093 21.90071)
4,5,【甘肃民乐：人工增雨全县普降甘霖】6月以来，我县气温特高，降水特少，最长连续无降水日达19天...,2023,8,24,甘肃省,张掖市,民乐县,海潮坝、双树寺人影作业点,甘肃省张掖市海潮坝,"100.592715,38.325645",住宅区,张掖市海潮坝,甘肃省张掖市民乐县海潮坝,2023-08-24,100.592715,38.325645,POINT (100.59272 38.32564)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2619,9559,【110发火箭弹炮弹齐上天 德州打下“春雨”】一场久违的春雨让德州市的旱情得到有效缓解。从4...,2016,4,15,山东省,德州市,None,22个作业点,山东省德州运河开发区,"116.266042,37.447177",住宅区,德州运河开发区,山东省德州市德城区运河开发区,2016-04-15,116.266042,37.447177,POINT (116.26604 37.44718)
2620,9560,【110发火箭弹炮弹齐上天 德州打下“春雨”】一场久违的春雨让德州市的旱情得到有效缓解。从4...,2016,4,15,山东省,德州市,None,22个作业点,山东省德州黄河涯乡,"116.346984,37.317352",乡镇,德州黄河涯乡,山东省德州市德城区黄河涯乡,2016-04-15,116.346984,37.317352,POINT (116.34698 37.31735)
2621,9561,【110发火箭弹炮弹齐上天 德州打下“春雨”】一场久违的春雨让德州市的旱情得到有效缓解。从4...,2016,4,15,山东省,德州市,None,22个作业点,山东省德州董子巷街,"116.382575,37.444296",道路,德州董子巷街,山东省德州市德城区董子巷,2016-04-15,116.382575,37.444296,POINT (116.38258 37.4443)
2622,9562,【110发火箭弹炮弹齐上天 德州打下“春雨”】一场久违的春雨让德州市的旱情得到有效缓解。从4...,2016,4,15,山东省,德州市,None,22个作业点,山东省德州马颊河沿岸区域,"116.421428,37.345536",住宅区,德州马颊河沿岸区域,山东省德州市德城区马颊河,2016-04-15,116.421428,37.345536,POINT (116.42143 37.34554)


In [85]:
year = 2023
month = 2
file_path = f"C:/Users/Anora/BFI Dropbox/Wanru Wu/MODIS/{year}/MODIS{year}{month:02}.dta"
temp_df = pd.read_stata(file_path)

In [86]:
temp_df

,cloud_optical_thickness,cloud_mask_fraction,latitude,longitude,year,month,day
0,NaN,0.039000,-89.5,-179.5,2023.0,2.0,28.0
1,NaN,0.002602,-88.5,-179.5,2023.0,2.0,28.0
2,100.201667,0.094369,-87.5,-179.5,2023.0,2.0,28.0
3,7.005000,0.200212,-86.5,-179.5,2023.0,2.0,28.0
4,62.939565,0.225674,-85.5,-179.5,2023.0,2.0,28.0
...,...,...,...,...,...,...,...
1814395,NaN,NaN,85.5,179.5,2023.0,2.0,1.0
1814396,NaN,NaN,86.5,179.5,2023.0,2.0,1.0
1814397,NaN,NaN,87.5,179.5,2023.0,2.0,1.0
1814398,NaN,NaN,88.5,179.5,2023.0,2.0,1.0


In [3]:
import pandas as pd
p1 = float('nan')
p2 = 2
p3 = 3
df = pd.DataFrame({"index":[1,2,2],"value":[p1,p2,p3]})
df.groupby(['index']).sum()


,value
index,
1,0.0
2,5.0


In [94]:

for index,row in gdf_pt.iterrows():
    print(row.coordinate)

100.179358,22.092365
100.406481,21.626384
100.390518,21.841447
100.290934,21.900714
100.592715,38.325645
107.510756,27.547389
106.705620,28.248106
107.932874,28.777846
107.040816,27.364160
102.331370,36.546470
100.809323,32.443336
101.060110,32.518518
101.073675,32.408801
101.871173,28.153419
117.829014,29.224345
117.685817,29.215357
119.736679,30.521939
119.780991,30.467556
119.762657,30.380214
119.806791,30.451613
120.034937,30.365257
120.189635,30.474795
119.975707,30.381605
119.945422,30.266365
116.026328,36.728470
116.117540,36.588189
116.323619,36.554164
115.665340,33.634162
115.778815,34.003463
115.986472,33.535538
116.194481,33.414389
115.921420,33.127694
116.148439,33.216217
116.560798,33.436797
116.485454,33.362540
116.323619,36.554164
116.147205,36.670531
116.026328,36.728470
115.941070,36.652387
116.117540,36.588189
116.322059,36.646269
116.258621,36.456761
102.366904,24.613859
102.323157,24.699313
102.515445,26.305177
102.414128,25.898842
102.447118,25.879202
102.414128,25